# Conexión por SSH

Inicializar la conexion ssh


In [1]:
import paramiko
import getpass

host = "158.251.88.197"
port = 22
username = "iaraya"
password = getpass.getpass()

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

# Multiple Container Loading Problem

In [2]:
from mclp import get_random_bin
from random import random 

def get_adjusted_vol(boxes):
    vol=0.0
    for box in boxes:
        vol += box.adj_vol*boxes[box]
    return vol

def get_vol(boxes):
    vol=0.0
    for box in boxes:
        vol += box.vol*boxes[box]
    return vol

def adjusted_swap(solution, n=2,  max_vol_accept=1.0, tolerance=0.1, verbose=True):

    bin_A = get_random_bin(s = solution)
    bin_B = get_random_bin(s = solution, b = bin_A)

    iniA = bin_A.adj_vol
    iniB = bin_B.adj_vol
    var_ini = (bin_A.adj_vol-1.0)**2 + (bin_B.adj_vol-1.0)**2

    nA = int(random.randint(1, n))
    boxes_A = bin_A.pop_random_boxes(nA)
    adjvolA = get_adjusted_vol(boxes_A)

    nB = int(random.randint(0, n))
    boxes_B = bin_B.pop_random_boxes(nB)
    adjvolB = get_adjusted_vol(boxes_B)

    r = tolerance*random.random()
    accepted = False

    if adjvolA > adjvolB and bin_B.adj_vol + adjvolA <= max_vol_accept + r:
        accepted = True
    elif adjvolB > adjvolA and  bin_A.adj_vol + adjvolB <= max_vol_accept + r:
        accepted = True

    if accepted:
        bin_A.insert_boxes(boxes_B)
        bin_B.insert_boxes(boxes_A)
    else: 
        return -10

    var_final = (bin_A.adj_vol-1.0)**2 + (bin_B.adj_vol-1.0)**2
    var_diff = var_final - var_ini

    return var_diff

def get_sorted_vols(solution):
    volumens = []
    acum_volu = 0

    for s in solution:
        s_boxes = s.boxes
        vol_boxes = get_vol(s_boxes)
        acum_volu += get_vol(s_boxes)
        volumens.append(vol_boxes)

    volumens.sort()

    return volumens

""" 
    Obtiene los bins ordenados por sus volumenes de menor a mayor
    [1,2,4,11,1,23,5,123] ---(sort) --> [1, 1, 2, 4, 5, 11, 23, 123]
    tip: Agregar reverse igual True para ordenarlas de mayor a menor
"""
def get_sorted_bins(solution):

    list_solution = list(solution)
    list_solution.sort(key=lambda x: x.vol)

    return list_solution


**Construcción de bin priorizando cajas (BSG)**

In [3]:
from bsg import (load_BRinstance,load_BRKGAinstance, load_LargeInstance, 
                load_productInstance, load_instances_elhedhli)
from mclp import generate_candidate_solution

inst =1
t_boxes = 100              # Cantidad de cajas
r_param = 1.5                 # Parametro de llenado del contenedor
min_fr = 0.99               # minimo porcentaje ocupado para considerar la generación de un bloque
max_bl = 10000              # maxima cantidad de bloques a generar por bsg
bsg_time = 5               # Tiempo de BSG
extra_args = '--max_bl={} --min_fr={}'.format(max_bl, min_fr)

'''
file_name_datalarge = f'../benchs/Data_Large/L_{t_boxes}/L_{t_boxes}_{inst}.txt'
L,W,H,boxes,id2box = load_LargeInstance(filename=file_name_datalarge)

'''
file_name_column = f"../benchs/Instance_CG/1000/bin_pack_instance_i(5)_c(1).txt"
L,W,H,boxes,id2box = load_instances_elhedhli(filename=file_name_column)


boxes=boxes.copy()
Vmax = L*W*H            # Volumen del contenedor

init_solution = generate_candidate_solution(ssh, L, W, H, boxes, id2box, r_param=r_param, bsg_time=bsg_time, extra_args=extra_args)

{'remaining': [[178, 1], [89, 1], [217, 1], [6, 2], [224, 1], [161, 1], [380, 2], [92, 1], [465, 1], [127, 1], [377, 1], [56, 4], [329, 1], [318, 1], [421, 2], [147, 1], [20, 1], [422, 3], [345, 1], [222, 1], [188, 1], [8, 1], [362, 1], [389, 1], [301, 1], [162, 2], [136, 2], [119, 3], [231, 2], [354, 2], [351, 2], [54, 1], [243, 1], [15, 1], [268, 1], [316, 1], [414, 1], [456, 1], [384, 3], [201, 1], [289, 1], [164, 1], [433, 1], [65, 2], [172, 1], [192, 3], [57, 3], [113, 2], [358, 2], [361, 2], [107, 1], [207, 2], [24, 1], [435, 2], [200, 1], [330, 1], [413, 1], [68, 1], [131, 1], [292, 1], [133, 1]], 'loaded': [[251, 1], [324, 4], [178, 3], [278, 2], [202, 3], [195, 1], [469, 5], [84, 8], [40, 2], [323, 3], [142, 1], [267, 2], [110, 3], [242, 1], [102, 8], [111, 1], [73, 6], [16, 1], [210, 4], [334, 2], [277, 1], [85, 1], [368, 1], [177, 3], [129, 5], [59, 1], [243, 1], [240, 4], [48, 2], [418, 2], [383, 2], [212, 1], [456, 1], [439, 4], [120, 2], [404, 3], [289, 4], [373, 5], [400

In [5]:
from IPython.display import Image
import importlib
import box_plotter
importlib.reload(box_plotter)


figure_list = []
dimension = [L,W,H]

bin1 = init_solution[0].layout

for bin in init_solution:
    bin_str = f'{bin.id}.png'
    box_dims = bin.layout
    
    
    if box_dims:
        box_plotter.save_container(dimension, box_dims, bin_str)
        figure_list.append(bin_str)

if not figure_list:
    print("No hay figuras para graficar")

for imageName in figure_list:
    display(Image(filename=imageName,width=200, height=300))

No hay figuras para graficar
